In [1]:
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import numpy as np 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision

from tqdm import tqdm, trange
import wandb

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
	from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def make_env(env_id, seed, idx, run_name): 
	def thunk(): 
		env = gym.make(env_id, render_mode='rgb_array')
		env = gym.wrappers.RecordEpisodeStatistics(env) 
		env = gym.wrappers.RecordVideo(env, f'videos/ppo/cartpole.mp4')
		# env.seed(seed) 
		env.action_space.seed(seed) 
		env.observation_space.seed(seed) 
		return env 
	return thunk() 

In [3]:
def layer_init(layer, std=np.sqrt(2), bias_const=0.0): 
	nn.init.orthogonal_(layer.weight, std) 
	nn.init.constant_(layer.bias, bias_const)
	return layer 

class Agent(nn.Module): 
	def __init__(self, envs): 
		super().__init__() 
		self.critic = nn.Sequential(
			layer_init(nn.Linear(np.array(envs.single_observation_space.shape).prod(), 64)), 
			nn.Tanh(), 
			layer_init(nn.Linear(64, 64)), 
			nn.Tanh(), 
			layer_init(nn.Linear(64, 1), std=1.0)
		)

		self.actor = nn.Sequential(
			layer_init(nn.Linear(np.array(envs.single_observation_space.shape).prod(), 64)), 
			nn.Tanh(), 
			layer_init(nn.Linear(64, 64)), 
			nn.Tanh(), 
			layer_init(nn.Linear(64, envs.single_action_space.n), std=0.01)
		)

	def get_value(self, x): 
		return self.critic(x) 
	
	def get_action_and_value(self, x, action=None): 
		logits = self.actor(x) 
		probs = torch.distributions.categorical.Categorical(logits=logits) 
		if action is None: 
			action = probs.sample() 
		return action, probs.log_prob(action), probs.entropy(), self.critic(x) 

In [4]:
wandb.init(
    project='CartPole PPO', 
    entity='uuzall', 
    sync_tensorboard=True, 
    name='the first', 
    monitor_gym=True, 
    save_code=True,
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: uuzall. Use `wandb login --relogin` to force relogin


In [4]:
writer = torch.utils.tensorboard.SummaryWriter(f'runs/the first')

AttributeError: module 'torch.utils' has no attribute 'tensorboard'

In [5]:
seed = 1337 
num_envs = 4
num_steps = 128
# temp = [make_env('CartPole-v1', seed + i, i, 'the first') for i in range(4)]

envs = gym.vector.SyncVectorEnv([
    lambda: make_env('CartPole-v1', seed+0, 0, 'the first'), 
    lambda: make_env('CartPole-v1', seed+1, 1, 'the first'), 
    lambda: make_env('CartPole-v1', seed+2, 2, 'the first'), 
    lambda: make_env('CartPole-v1', seed+3, 3, 'the first')
])

agent = Agent(envs).to(device) 
optimizer = optim.Adam(agent.parameters(), lr=2.5e-4, eps=1e-5)

x:\python_environments\AI_310\lib\site-packages\gymnasium\wrappers\record_video.py:87: UserWarning: WARN: Overwriting existing videos at c:\Users\Asus\OneDrive\Programming\basic_rl\videos\ppo\cartpole.mp4 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [6]:
obs = torch.zeros((num_steps, num_envs) + envs.single_observation_space.shape).to(device)
actions = torch.zeros((num_steps, num_envs) + envs.single_action_space.shape).to(device) 
logprobs = torch.zeros((num_steps, num_envs)).to(device) 
rewards = torch.zeros((num_steps, num_envs)).to(device) 
dones = torch.zeros((num_steps, num_envs)).to(device) 
values = torch.zeros((num_steps, num_envs)).to(device) 

In [11]:
num_steps, num_envs, envs.single_action_space.shape

(128, 4, ())

In [8]:
total_timesteps = 64#*4096
bs = 64
gamma = 0.99
update_epochs = 4 
mini_bs = 4 
clip_coef = 0.2
ent_coef = 0.01 
vf_coef = 0.5

global_step = 0 
next_obs = torch.tensor(envs.reset()[0]).to(device) 
next_done = torch.zeros(num_envs).to(device) 
num_updates = total_timesteps // bs

for update in trange(1, num_updates + 1): 
	for step in range(0, num_steps): 
		global_step += 1 * num_envs 
		obs[step] = next_obs 
		dones[step] = next_done 
		
		with torch.no_grad(): 
			action, logprob, _, value = agent.get_action_and_value(next_obs) 
			values[step] = value.flatten() 
		actions[step] = action 
		logprobs[step] = logprob

		next_obs, reward, terminated, truncated, info = envs.step(action.cpu().numpy())
		done = np.bitwise_or(terminated, truncated).astype(np.float32) 
		rewards[step] = torch.tensor(reward).to(device).view(-1) 
		next_obs, next_done = torch.tensor(next_obs).to(device), torch.tensor(done).to(device) 

	with torch.no_grad(): 
		next_value = agent.get_value(next_obs).reshape(1, -1) 
		# no GAE 	
		returns = torch.zeros_like(rewards).to(device) 
		for t in reversed(range(num_steps)): 
			if t == num_steps - 1: 
				next_non_terminal = 1.0 - next_done 
				next_return = next_value 
			else: 
				next_non_terminal = 1.0 - dones[t + 1]
				next_return = returns[t + 1]
			returns[t] = rewards[t] + gamma * next_non_terminal * next_return 
		advantages = returns - values 

	b_obs = obs.reshape((-1,) + envs.single_observation_space.shape) 
	b_logprobs = logprobs.reshape(-1) 
	b_actions = actions.reshape((-1,) + envs.single_action_space.shape)
	b_advantages = advantages.reshape(-1) 
	b_returns = returns.reshape(-1) 
	b_values = values.reshape(-1) 

	b_inds = np.arange(bs) 
	clipfracs = list() 
	
	for epoch in range(update_epochs): 
		np.random.shuffle(b_inds)
		for start in range(0, bs, mini_bs): 
			end = start + mini_bs 
			mb_inds = b_inds[start:end]

			_, newlogprob, entropy, newvalue = agent.get_action_and_value(b_obs[mb_inds], b_actions.long()[mb_inds])
			logratio = newlogprob - b_logprobs[mb_inds]
			ratio = logratio.exp() 

			with torch.no_grad(): 
				old_approx_kl = (-logprob).mean() 
				approx_kl = ((ratio - 1) - logratio).mean() 
				clipfracs += [((ratio - 1.0).abs() > clip_coef).float().mean().item()]
			
			mb_advantages = b_advantages[mb_inds]
			# normalizing advantages can be turned off 
			mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + 1e-8)

			pg_loss1 = -mb_advantages * ratio 
			pg_loss2 = -mb_advantages * torch.clamp(ratio, 1 - clip_coef, 1 + clip_coef) 
			pg_loss = torch.max(pg_loss1, pg_loss2).mean() 

			v_loss_unclipped = (newvalue - b_returns[mb_inds])**2 
			v_clipped = b_values[mb_inds] + torch.clamp(newvalue - b_values[mb_inds], -clip_coef, clip_coef)
			v_loss_clipped = (v_clipped - b_returns[mb_inds])**2 
			v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped) 
			v_loss = 0.5 * v_loss_max.mean() 

			entropy_loss = entropy.mean() 
			loss = pg_loss - ent_coef * entropy_loss + v_loss * vf_coef 

			optimizer.zero_grad() 
			loss.backward() 
			nn.utils.clip_grad_norm_(agent.parameters(), 0.5) 
			optimizer.step() 

		y_pred, y_true = b_values.cpu().numpy(), b_returns.cpu().numpy() 
		var_y = np.var(y_true) 
		explained_var = np.nan if var_y == 0 else 1 - np.var(y_true - y_pred) / var_y 

		writer.add_scalar('charts/learning_rate', optimizer.param_groups[0]['lr'], global_step)
		writer.add_scalar('losses/value_loss', v_loss.item(), global_step)
		writer.add_scalar("losses/policy_loss", pg_loss.item(), global_step)
		writer.add_scalar("losses/entropy", entropy_loss.item(), global_step)
		writer.add_scalar("losses/old_approx_kl", old_approx_kl.item(), global_step)
		writer.add_scalar("losses/approx_kl", approx_kl.item(), global_step)
		writer.add_scalar("losses/clipfrac", np.mean(clipfracs), global_step)
		writer.add_scalar("losses/explained_variance", explained_var, global_step)

  0%|          | 0/4096 [00:02<?, ?it/s]

Moviepy - Building video c:\Users\Asus\OneDrive\Programming\basic_rl\videos\ppo\cartpole.mp4\rl-video-episode-0.mp4.
Moviepy - Writing video c:\Users\Asus\OneDrive\Programming\basic_rl\videos\ppo\cartpole.mp4\rl-video-episode-0.mp4



  0%|          | 0/4096 [00:02<?, ?it/s]

Moviepy - Done !
Moviepy - video ready c:\Users\Asus\OneDrive\Programming\basic_rl\videos\ppo\cartpole.mp4\rl-video-episode-0.mp4
Moviepy - Building video c:\Users\Asus\OneDrive\Programming\basic_rl\videos\ppo\cartpole.mp4\rl-video-episode-0.mp4.
Moviepy - Writing video c:\Users\Asus\OneDrive\Programming\basic_rl\videos\ppo\cartpole.mp4\rl-video-episode-0.mp4



  0%|          | 0/4096 [00:02<?, ?it/s]

...

 76%|███████▌  | 3106/4096 [42:24<16:37,  1.01s/it]

Moviepy - Building video c:\Users\Asus\OneDrive\Programming\basic_rl\videos\ppo\cartpole.mp4\rl-video-episode-2000.mp4.
Moviepy - Writing video c:\Users\Asus\OneDrive\Programming\basic_rl\videos\ppo\cartpole.mp4\rl-video-episode-2000.mp4



 76%|███████▌  | 3106/4096 [42:24<16:37,  1.01s/it]

Moviepy - Done !
Moviepy - video ready c:\Users\Asus\OneDrive\Programming\basic_rl\videos\ppo\cartpole.mp4\rl-video-episode-2000.mp4


100%|██████████| 4096/4096 [55:21<00:00,  1.23it/s]


In [10]:
torch.save(agent.state_dict(), 'models/cartpole_ppo/4096_train.pth')